In [35]:
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np

In [36]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [37]:
df = pd.read_csv(r'..\No_sync\analyst_ratings_processed_filtered.csv')
sentences = df['title'].tolist()

In [42]:
inputs = tokenizer(sentences, return_tensors="pt", padding=True)
outputs = finbert(**inputs)[0]

# Define a softmax function
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=0)

# Define labels for reference
labels = {0: 'neutral', 1: 'positive', 2: 'negative'}

import pandas as pd
import numpy as np

# Your existing code for sentences, inputs, outputs, softmax, and labels

# Create an empty list to store data
data = []

for idx, sent in enumerate(sentences):
    probabilities = softmax(outputs.detach().numpy()[idx])
    row_data = [sent] + [f'{prob * 100:.2f}%' for prob in probabilities]
    data.append(row_data)

# Define column names for the DataFrame
columns = ['title'] + [f'{label} Probability' for label in labels.values()]

# Create a DataFrame
new_df = pd.DataFrame(data, columns=columns)


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1889823744 bytes.

In [ ]:
df = df.merge(new_df, on='title', how='left')
df.shape()

,Unnamed: 0,title,date,stock,neutral Probability,positive Probability,negative Probability
0,7120.0,Tech Stocks And FAANGS Strong Again To Start D...,2020-06-10 11:33:00-04:00,AAPL,0.00%,100.00%,0.00%
1,7121.0,10 Biggest Price Target Changes For Wednesday,2020-06-10 08:14:00-04:00,AAPL,100.00%,0.00%,0.00%
2,7122.0,"Benzinga Pro's Top 5 Stocks To Watch For Wed.,...",2020-06-10 07:53:00-04:00,AAPL,100.00%,0.00%,0.00%
3,7123.0,"Deutsche Bank Maintains Buy on Apple, Raises P...",2020-06-10 07:19:00-04:00,AAPL,0.00%,100.00%,0.00%
4,7124.0,Apple To Let Users Trade In Their Mac Computer...,2020-06-10 06:27:00-04:00,AAPL,100.00%,0.00%,0.00%
5,7125.0,Big Tech Reaches New Record Heights At The Sto...,2020-06-10 00:52:00-04:00,AAPL,0.02%,99.98%,0.00%
6,7126.0,Why Apple's Stock Is Trading Higher Today,2020-06-09 15:14:00-04:00,AAPL,0.00%,100.00%,0.00%
7,7127.0,Apple Could Announce In-House Chips For Macs A...,2020-06-09 13:58:00-04:00,AAPL,99.99%,0.01%,0.00%
8,7128.0,Apple shares are trading higher despite market...,2020-06-09 12:41:00-04:00,AAPL,0.00%,100.00%,0.00%
9,7129.0,"Sonos Shares Spike To Session High, Now Up 9.5...",2020-06-09 11:11:00-04:00,AAPL,8.82%,91.16%,0.03%
